In [1]:
import os
import time

import tensorflow as tf
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import tensorflow_probability as tfp
from tensorflow.keras import backend as K
import pdb

In [2]:
data_directories = ["../DATA/b_cells_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/cd4_t_helper_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/cd14_monocytes_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/cd34_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/cd56_nk_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/cytotoxic_t_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/memory_t_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/naive_cytotoxic_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/naive_t_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/",
                    "../DATA/regulatory_t_filtered_gene_bc_matrices/filtered_matrices_mex/hg19/"]
cell_types = ['B_cell','CD4+_T_cell','Myeloid','Myeloid','NK_cell','CD8+_T_cell','CD4+_T_cell','CD8+_T_cell','CD4+_T_cell','Treg']
bkdata_paths = ['../DATA/TCGA/TCGA_GDC_HTSeq_TPM.csv',
              '../DATA/METABRIC/METABRIC.csv',
              '../DATA/SDY67/SDY67_477.csv']
# gene_list_path = '../DATA/Immune Gene Lists/genes.csv'
data_paths = ['../DATA/TCGA/TCGA_GDC_HTSeq_TPM.csv',
              '../DATA/METABRIC/METABRIC.csv',
              '../DATA/SDY67/SDY67_477.csv',
              '../DATA/Gene Lists/immport_genelist.csv',
              '../DATA/Gene Lists/scdata_genelist_filtered_v2.csv']

#### Define functions for data processing

In [3]:
def FeatureList(paths: list) -> list:
    features = None
    for path in tqdm(paths):
        mydata = pd.read_csv(path, index_col = 0)
        if features == None:
            features = set(mydata.index.values.tolist())
        else:
            features = features.intersection(set(mydata.index.values.tolist()))
    features = list(features)
    features.sort()
    return features

def MinMaxNorm(x):
    x_scaled = tf.math.divide_no_nan(
        (x - tf.math.reduce_min(x)),
        (tf.math.reduce_max(x) - tf.math.reduce_min(x)))
    return x_scaled

#### Data preprocessing

In [4]:
'''
scdata should be in matrix.mtx (or matrix.mtx.gz) within specified folders along with barcodes.tsv and genes.tsv
bkdata should have sample names as columns and gene names as rows
'''

# Select features
print('Loading datasets to select features')
features = FeatureList(data_paths)

Loading datasets to select features


In [5]:
## Read Wu EMBO metadata
print('Loading Wu EMBO Breast Cancer scRNA-seq dataset')
x_meta = pd.read_csv("../DATA/TNBC/Wu_EMBO_metadata_v2.csv", index_col=0)
## Read Wu EMBO data (Breast Cancer)
scdata = sc.read_10x_mtx("../DATA/TNBC/counts_matrix/")
## Add cell type annotations from metadata into .obs
scdata.obs = pd.DataFrame(scdata.obs.merge(x_meta, left_index=True, right_index=True).loc[:,['celltype_final','patientID']])
scdata.obs.columns = ['celltype', 'patientID'] # rename column
scdata.obs.set_index(pd.Index([c+'_'+rn[-16:-1] for c, rn in zip(scdata.obs.celltype, scdata.obs.index)]), inplace=True)

## Read and merge 10X Genomics scRNA-seq data
print('Loading Zheng 10X Genomics PBMC scRNA-seq datasets')
for d, c in zip(tqdm(data_directories), cell_types):
    x = sc.read_10x_mtx(d)
    x.obs['celltype'] = [c]*len(x.obs.index)
    # Change each observation (cell) name to celltype + barcode
    x.obs.set_index(pd.Index([c+'_'+rn[:-2] for rn in x.obs.index]), inplace=True)
    if scdata is not None:
        scdata = ad.concat([scdata, x])
    else:
        scdata = x

# Filter out cells and genes
print('Filtering and preprocessing scRNA-seq datasets')
sc.pp.filter_cells(scdata, min_genes=200)
sc.pp.filter_genes(scdata, min_cells=1)
# Search for prefix "MT-" (mitochondrial genes) and make new column in variable annotations
# Search for prefix "RPL/RPS" for ribosomal genes and "MRPL/MRPS" for mitochondrial ribosomal genes
scdata.var['mito'] = scdata.var.index.str.match('^MT-')
scdata.var['ribo'] = scdata.var.index.str.startswith(('RPL','RPS'))
scdata.var['mribo'] = scdata.var.index.str.startswith(('MRPL','MRPS'))
# Calculate QC metrics as per McCarthy et al., 2017 (Scater)
sc.pp.calculate_qc_metrics(scdata, qc_vars=['mito','ribo', 'mribo'], inplace=True)
# Filter out cells with >5% of counts from mitochondria and mitoribosome
# scdata = scdata[scdata.obs.pct_counts_ribo > 30, :]
scdata = scdata[scdata.obs.pct_counts_mito < 5, :]
scdata = scdata[scdata.obs.pct_counts_mribo < 1, :]
# Filter out genes not in gene list for scdata and subdivide into list of datasets for each cell type
scdata = scdata[:,scdata.var_names.isin(features)]
sc.pp.normalize_total(scdata, target_sum=1e6) # normalize to sum to 1,000,000
# sc.pp.regress_out(scdata, ['total_counts'], n_jobs=1) # takes too long to complete
print('Dividing single cell dataset into cell types')
scdata_ = []
for c in tqdm(scdata.obs.celltype.unique().tolist()):
    scdata_.append(scdata[scdata.obs.celltype==c].to_df().sort_index(axis=1))

Loading Wu EMBO Breast Cancer scRNA-seq dataset
Loading Zheng 10X Genomics PBMC scRNA-seq datasets


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.



Filtering and preprocessing scRNA-seq datasets


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
c:\users\yw_ji\documents\msc thesis\code\keras\scanpy\scanpy\preprocessing\_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


Dividing single cell dataset into cell types


In [6]:
# Load bulk RNA-seq dataset (TCGA-BRCA)
bkdata = []
for d in tqdm(bkdata_paths):
    print(f'Loading bulk dataset: {d}')
    x = pd.read_csv(d, index_col=0)
    x = x.dropna(axis=1)
    print('Processing bulk dataset')
    # Transpose, filter out genes not in gene list, then sort column (by gene name)
    x = x.T
    x = x.loc[:,x.columns.isin(features)].sort_index(axis=1)
    x = x.values.astype(float)
    x = MinMaxNorm(tf.math.log1p(x))
    bkdata.append(x)
    
bkdata = tf.concat(bkdata, axis=0)

Loading bulk dataset: ../DATA/TCGA/TCGA_GDC_HTSeq_TPM.csv
Processing bulk dataset
Loading bulk dataset: ../DATA/METABRIC/METABRIC.csv
Processing bulk dataset
Loading bulk dataset: ../DATA/SDY67/SDY67_477.csv
Processing bulk dataset



#### Define new layers

In [7]:
class Subsampling(tf.keras.layers.Layer):

    def __init__(self, scdata):
        super(Subsampling, self).__init__()
        # initialize one layer for each cell type
        while scdata.shape[0] < 501:
            scdata = pd.concat([scdata, scdata])
        self.scdata=scdata

    def call(self, inpt):
        return tf.reduce_sum(tf.slice(tf.random.shuffle(self.scdata), begin=[0,0], size=[inpt.numpy(), -1]), axis=0)

#### Define model architecture

In [8]:
class SimulateFractions():
    
    def __init__(self, scdata):
        self.scdata = scdata
        self.n_celltypes = len(scdata)
    
    def __call__(self, x, nprop):
        dist = tfp.distributions.DirichletMultinomial(total_count=[500]*len(x), concentration=x)
        x = dist.sample() # no argument to ".sample()" produces output of shape (batchsize, # of classes)
        x = tf.cast(x, dtype=tf.int32) # "tf.slice" takes in tensor of integer values
        nprop.append(x)
        
        x_=[]
        for c in range(self.n_celltypes):
            x_.append(Subsampling(self.scdata[c])(x[0][c]))
        x=tf.keras.layers.Add()(x_)
        return x

In [9]:
class Build_Normalizer(tf.keras.Model):

    def __init__(self, h_size):
        super(Build_Normalizer, self).__init__()
        self.Noise = tf.keras.layers.GaussianNoise(12.0)
        self.Dense = tf.keras.layers.Dense(h_size)
        self.Expand = tf.keras.layers.Reshape((1,h_size), input_shape=(h_size,))
        self.Pool = tf.keras.layers.AveragePooling1D(3, strides=1, padding='same')
        self.Flat = tf.keras.layers.Flatten()
#         self.Activation = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.Activation = tf.keras.layers.Activation(tf.keras.activations.softplus)
        self.Log1p =tf.keras.layers.Lambda(lambda t: tf.math.log1p(t))
        self.Norm = tf.keras.layers.LayerNormalization(center=True, scale=True)
        self.MinMax = tf.keras.layers.Lambda(lambda t: MinMaxNorm(t))

    def call(self,x):
#         x=self.Noise(x)
        x=self.Dense(x)
        x=self.Activation(x)
        x=self.Expand(x)
        x=self.Pool(x)
        x=self.Flat(x)
#         x=self.Dense(x)
#         x=self.Activation(x)
#         x=self.Log1p(x)
        x=self.Norm(x)
        x=self.MinMax(x)
        return x

In [10]:
class Build_Discriminator(tf.keras.Model):

    def __init__(self):
        super(Build_Discriminator, self).__init__()
        self.Noise = tf.keras.layers.GaussianNoise(16.0)
        self.Dense1 = tf.keras.layers.Dense(12)
        self.Activation = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.Drop = tf.keras.layers.Dropout(0.3)
#         self.Dense2 = tf.keras.layers.Dense(12)
        self.Output = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self,x):
        x=self.Noise(x)
        x=self.Dense1(x)
        x=self.Activation(x)
        x=self.Drop(x)
#         x=self.Dense2(x)
#         x=self.Activation(x)
#         x=self.Drop(x)
        x=self.Output(x)
        return x

#### Instantiate models

In [11]:
optmzr = tf.keras.optimizers.Adam(0.0002, 0.5)

discriminator = Build_Discriminator()
discriminator.compile(loss=tf.losses.BinaryCrossentropy(label_smoothing=0.2), 
                     optimizer=optmzr,
                     metrics=['accuracy'])

simulator = SimulateFractions(scdata_)
normalizer = Build_Normalizer(scdata_[0].shape[1])
z = tf.keras.layers.Input(shape=(scdata_[0].shape[1],))
simbulk = normalizer(z)

discriminator.trainable = False
validity = discriminator(simbulk)

model = tf.keras.Model(z, validity)
model.compile(loss='binary_crossentropy', optimizer=optmzr, metrics=['accuracy'])

#### TRAIN

In [15]:
STEPS = 100
BATCH_SIZE = 32
SAMPLE_INTERVAL = 5
N_CELLTYPE = len(scdata_)

nprop_sim = []
nprop_adv = []

# Ground truths
valid = np.ones((BATCH_SIZE,1))
fake = np.zeros((BATCH_SIZE,1))

for step in tqdm(tf.range(STEPS)):
    # Train Discriminator
    real = tf.random.shuffle(bkdata)[0:BATCH_SIZE]
    
    inpts = []
    for n in tf.range(BATCH_SIZE):
        noise = tf.random.uniform([1,N_CELLTYPE], 0,1)
        inpt = simulator(noise, nprop_sim)
        inpts.append(inpt)
    
    gen_bulk = normalizer(tf.stack(inpts))
    
    d_loss_real = discriminator.train_on_batch(real, valid)
    d_loss_fake = discriminator.train_on_batch(gen_bulk, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train Normalizer as part of the entire adversarial model
    inpts = []
    for n in tf.range(BATCH_SIZE):
        noise = tf.random.uniform([1,N_CELLTYPE], 0,1)
        inpt = simulator(noise, nprop_adv)
        inpts.append(inpt)
    
    n_loss = model.train_on_batch(tf.stack(inpts), valid)
    
    # Progress updates
    print ("STEP: %d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%%]" % (step, d_loss[0], 100*d_loss[1], n_loss[0], 100*(1-n_loss[1])))
    
    if step % SAMPLE_INTERVAL == 0:
        sample_images(step, N_CELLTYPE)

STEP: 0 [D loss: 13.692750, acc.: 53.12%] [G loss: 20.540020, acc.: 65.62%]
STEP: 1 [D loss: 13.771379, acc.: 42.19%] [G loss: 9.462070, acc.: 46.88%]
STEP: 2 [D loss: 8.196848, acc.: 56.25%] [G loss: 11.606221, acc.: 50.00%]
STEP: 3 [D loss: 14.508796, acc.: 31.25%] [G loss: 15.654581, acc.: 56.25%]
STEP: 4 [D loss: 11.280535, acc.: 39.06%] [G loss: 8.134229, acc.: 46.88%]
STEP: 5 [D loss: 13.156456, acc.: 46.88%] [G loss: 11.427497, acc.: 50.00%]
STEP: 6 [D loss: 10.647680, acc.: 50.00%] [G loss: 4.598886, acc.: 31.25%]
STEP: 7 [D loss: 10.567724, acc.: 42.19%] [G loss: 7.251333, acc.: 40.62%]
STEP: 8 [D loss: 12.104508, acc.: 37.50%] [G loss: 9.736481, acc.: 43.75%]
STEP: 9 [D loss: 12.333277, acc.: 46.88%] [G loss: 2.436189, acc.: 28.12%]
STEP: 10 [D loss: 11.422166, acc.: 42.19%] [G loss: 14.588655, acc.: 59.38%]
STEP: 11 [D loss: 18.277048, acc.: 35.94%] [G loss: 6.729623, acc.: 46.88%]
STEP: 12 [D loss: 12.944206, acc.: 40.62%] [G loss: 7.368390, acc.: 50.00%]
STEP: 13 [D loss: 

In [16]:
nprop = []
noise = tf.random.uniform([1,N_CELLTYPE], 0,1)
inpt = simulator(noise, nprop)
    
print(model.predict(tf.expand_dims(inpt, axis=0)))

print(model.predict(tf.expand_dims(tf.random.shuffle(bkdata)[0], axis=0)))

[[0.9958269]]
[[0.99400973]]


In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3000)]            0         
_________________________________________________________________
build__normalizer (Build_Nor (None, 3000)              9009000   
_________________________________________________________________
build__discriminator (Build_ (None, 1)                 36025     
Total params: 9,045,025
Trainable params: 9,009,000
Non-trainable params: 36,025
_________________________________________________________________


In [ ]:
model.layers[1].summary()

In [41]:
nprop

[<tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[  6,  95,   0, 161,  27,   0, 121,   0,  62,   0,  17,  11]])>]

In [45]:
nprop = []

inpts = []
for n in tf.range(100000):
    noise = tf.random.uniform([1,N_CELLTYPE], 0,1)
    inpt = simulator(noise, nprop)
    inpts.append(inpt)
    if n % 1000 == 0:
        print(n)

gen_bulk = model.layers[1].predict(tf.stack(inpts))


tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1000, shape=(), dtype=int32)
tf.Tensor(2000, shape=(), dtype=int32)
tf.Tensor(3000, shape=(), dtype=int32)
tf.Tensor(4000, shape=(), dtype=int32)
tf.Tensor(5000, shape=(), dtype=int32)
tf.Tensor(6000, shape=(), dtype=int32)
tf.Tensor(7000, shape=(), dtype=int32)
tf.Tensor(8000, shape=(), dtype=int32)
tf.Tensor(9000, shape=(), dtype=int32)
tf.Tensor(10000, shape=(), dtype=int32)
tf.Tensor(11000, shape=(), dtype=int32)
tf.Tensor(12000, shape=(), dtype=int32)
tf.Tensor(13000, shape=(), dtype=int32)
tf.Tensor(14000, shape=(), dtype=int32)
tf.Tensor(15000, shape=(), dtype=int32)
tf.Tensor(16000, shape=(), dtype=int32)
tf.Tensor(17000, shape=(), dtype=int32)
tf.Tensor(18000, shape=(), dtype=int32)
tf.Tensor(19000, shape=(), dtype=int32)
tf.Tensor(20000, shape=(), dtype=int32)
tf.Tensor(21000, shape=(), dtype=int32)
tf.Tensor(22000, shape=(), dtype=int32)
tf.Tensor(23000, shape=(), dtype=int32)
tf.Tensor(24000, shape=(), dtype=int32)
tf.Tensor(250

In [50]:
np.savetxt('../DATA/simbulk/210319_GAN_N100000_C500_simbulk_data.csv', gen_bulk, delimiter=',')

In [63]:
nprop = tf.squeeze(tf.stack(nprop))

In [70]:
nprop_sum

<tf.Tensor: shape=(100000,), dtype=int32, numpy=array([500, 500, 500, ..., 500, 500, 500])>

In [72]:
nprop_sum = nprop/500
nprop_sum

<tf.Tensor: shape=(100000, 12), dtype=float64, numpy=
array([[0.008, 0.   , 0.006, ..., 0.308, 0.05 , 0.004],
       [0.128, 0.   , 0.   , ..., 0.024, 0.228, 0.084],
       [0.   , 0.   , 0.104, ..., 0.   , 0.144, 0.096],
       ...,
       [0.004, 0.012, 0.008, ..., 0.   , 0.182, 0.036],
       [0.146, 0.   , 0.14 , ..., 0.062, 0.532, 0.024],
       [0.358, 0.044, 0.136, ..., 0.   , 0.014, 0.   ]])>

In [62]:
np.savetxt('../DATA/simbulk/210319_GAN_N100000_C500_simbulk_label.csv', tf.squeeze(tf.stack(nprop)), delimiter=',')

In [61]:
np.savetxt('../DATA/simbulk/210319_Simulator_N100000_C500_simbulk_data.csv', )

TensorShape([3, 12])

In [73]:
np.savetxt('../DATA/simbulk/210319_GAN_N100000_C500_simbulk_label.csv', nprop_sum, delimiter=',')

In [75]:
model.save('./log/models/210319/')

INFO:tensorflow:Assets written to: ./log/models/assets


In [77]:
new_model = tf.keras.models.load_model('./log/models/210319/')
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3000)]            0         
_________________________________________________________________
build__normalizer (Build_Nor (None, 3000)              9009000   
_________________________________________________________________
build__discriminator (Build_ (None, 1)                 36025     
Total params: 9,045,025
Trainable params: 9,045,025
Non-trainable params: 0
_________________________________________________________________


In [13]:
def sample_images(epoch, n_celltype):
    r, c = 5,5
    inpts = []
    for n in tf.range(r*c):
        noise = tf.random.uniform([1,n_celltype], 0,1)
        inpt = simulator(noise, nprop_adv)
        inpts.append(inpt)
    gen_imgs = normalizer.predict(tf.stack(inpts))
    
    feature_len = gen_imgs.shape[1]
    target_len = min([i**2 for i in range(100) if i**2 > feature_len])
    target_dim = int(target_len**(0.5))
    
    fig, axs = plt.subplots(r,c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(np.append(gen_imgs[cnt], [0]*(target_len - feature_len)).reshape(target_dim, target_dim), cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig('log/images/%d.png' % epoch)
    plt.close()
            

In [14]:
def bulk_image(bkdata):
    r, c = 5,5
    
    feature_len = bkdata.shape[1]
    target_len = min([i**2 for i in range(100) if i**2 > feature_len])
    target_dim = int(target_len**(0.5))
    
    fig, axs = plt.subplots(r,c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(np.append(bkdata[cnt], [0]*(target_len - feature_len)).reshape(target_dim, target_dim), cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig('log/images/bulk.png')
    plt.close()
            

In [10]:
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
dml = tfp.layers.DistributionLambda(
    make_distribution_fn=lambda t: tfp.distributions.DirichletMultinomial(
        total_count=500, concentration=t),
    convert_to_tensor_fn=lambda s: s.sample()
)